In [ ]:
pip install requests pandas numpy


Analyze historical interaction data from Compound V2 or V3 for a list of wallet addresses. Use that data to develop a quantitative risk scoring system that assigns each wallet a score from 0 (high risk) to 1000 (low risk).
Step 1: Data Collection
✅ Goal:
Fetch each wallet’s interaction with the Compound protocol:

Supply (cToken mints)

Redeem (withdrawals)

Borrow

Repay

Liquidation events

📡 Data Sources:
Use Compound’s official subgraphs hosted on The Graph:

Compound V2 Subgraph: https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2

Compound V3 (Comet) Subgraphs vary by chain (e.g. Ethereum, Arbitrum, etc.)
Step 2: Data Preparation
For each wallet, compute these features to capture risk behavior:

Feature	Definition	Risk Implication
net_borrow_to_supply_ratio	total_borrowed / total_supplied	High ratio → High risk
repayment_ratio	total_repaid / total_borrowed	Low ratio → High risk
liquidation_count	Number of liquidation events	More liquidations → Higher risk
avg_days_between_tx	Inactivity periods between Compound txs	Irregularity = Higher risk
unique_assets	Number of markets used	Low diversity = Higher risk
protocol_tenure_days	Days since first Compound tx	Newer users = Higher risk

🧮 Step 3: Risk Scoring Model (0–1000)
🔍 Normalize All Features
Using min-max scaling:

python
Copy
Edit
normalized = (value - min_value) / (max_value - min_value)
🧩 Weighted Composite Score
Assign feature weights (these are tunable):

Feature	Weight
Borrow/Supply Ratio	30%
Repayment Ratio	25%
Liquidation Count	20%
Protocol Tenure	10%
Asset Diversity	10%
Avg Tx Frequency	5%

Use the formula:

python
Copy
Edit
risk_score = 1000 * (1 - weighted_risk_index)
Where:

weighted_risk_index is the weighted average of normalized high-risk indicators.

 Step 4: Final Output
You will generate a CSV in the format:

wallet_id	score
0xabc...	732
0xdef...	914

 Sample Feature Table (Mock Data)
wallet_id	borrow_supply_ratio	repay_ratio	liquidations	score
0x123...	0.82	0.65	2	443
0xabc...	0.18	0.95	0	910

 Scalability Consideration
Add new wallets dynamically by re-querying the subgraph.

You can adapt to Aave, Morpho, or other protocols by plugging into their subgraphs.



In [ ]:
import requests
import pandas as pd
import numpy as np


In [ ]:
def fetch_wallet_data(wallet_address):
    url = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"
    query = """
    {
      account(id: "%s") {
        id
        tokens {
          symbol
          supplyBalanceUnderlying
          borrowBalanceUnderlying
        }
      }
      liquidations(where: {borrower: "%s"}) {
        id
      }
    }
    """ % (wallet_address.lower(), wallet_address.lower())

    response = requests.post(url, json={'query': query})
    data = response.json()
    return data


In [ ]:
def extract_features(wallet_address):
    data = fetch_wallet_data(wallet_address)

    # Check if 'data' key exists in the response
    if 'data' not in data or data['data'] is None:
        print(f"Warning: Could not fetch data for wallet {wallet_address}")
        return {
            'wallet_id': wallet_address,
            'total_supply': 0,
            'total_borrow': 0,
            'borrow_to_supply': 0,
            'liquidation_count': 0
        }

    account = data['data'].get('account')
    liquidations = data['data'].get('liquidations', [])

    supply = 0
    borrow = 0

    if account and account.get('tokens'):
        for token in account['tokens']:
            supply += float(token.get('supplyBalanceUnderlying', 0))
            borrow += float(token.get('borrowBalanceUnderlying', 0))

    liquidation_count = len(liquidations)

    features = {
        'wallet_id': wallet_address,
        'total_supply': supply,
        'total_borrow': borrow,
        'borrow_to_supply': borrow / supply if supply > 0 else 0,
        'liquidation_count': liquidation_count
    }
    return features

In [ ]:
def fetch_wallet_data(wallet_address):
    url = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"
    query = """
    {
      account(id: "%s") {
        id
        tokens {
          symbol
          supplyBalanceUnderlying
          borrowBalanceUnderlying
        }
      }
      liquidations(where: {borrower: "%s"}) {
        id
      }
    }
    """ % (wallet_address.lower(), wallet_address.lower())

    response = requests.post(url, json={'query': query})

    try:
        data = response.json()
        if 'data' in data:
            return data['data']
        else:
            print(f"[ERROR] GraphQL query failed for {wallet_address}: {data.get('errors', 'Unknown error')}")
            return None
    except Exception as e:
        print(f"[EXCEPTION] Could not parse JSON for {wallet_address}: {e}")
        return None


In [ ]:
def extract_features(wallet_address):
    data = fetch_wallet_data(wallet_address)
    if not data:
        return {
            'wallet_id': wallet_address,
            'total_supply': 0,
            'total_borrow': 0,
            'borrow_to_supply': 0,
            'liquidation_count': 0
        }

    account = data.get('account')
    liquidations = data.get('liquidations', [])

    supply = 0
    borrow = 0

    if account and 'tokens' in account:
        for token in account['tokens']:
            supply += float(token['supplyBalanceUnderlying'])
            borrow += float(token['borrowBalanceUnderlying'])

    features = {
        'wallet_id': wallet_address,
        'total_supply': supply,
        'total_borrow': borrow,
        'borrow_to_supply': borrow / supply if supply > 0 else 0,
        'liquidation_count': len(liquidations)
    }
    return features


In [ ]:
print(extract_features("0x0039f22efb07a647557c7c5d17854cfd6d489ef3"))


[ERROR] GraphQL query failed for 0x0039f22efb07a647557c7c5d17854cfd6d489ef3: [{'message': 'This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com'}]
{'wallet_id': '0x0039f22efb07a647557c7c5d17854cfd6d489ef3', 'total_supply': 0, 'total_borrow': 0, 'borrow_to_supply': 0, 'liquidation_count': 0}


In [ ]:
features_list = [extract_features(wallet) for wallet in wallets]
df = pd.DataFrame(features_list)


[ERROR] GraphQL query failed for 0x0039f22efb07a647557c7c5d17854cfd6d489ef3: [{'message': 'This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com'}]
[ERROR] GraphQL query failed for 0x06b51c6882b27cb05e712185531c1f74996dd988: [{'message': 'This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com'}]


In [ ]:
import requests
import pandas as pd
import numpy as np
import time

# ---------- Step 1: List of Wallets ----------
wallets = [
    "0x0039f22efb07a647557c7c5d17854cfd6d489ef3",
    "0x06b51c6882b27cb05e712185531c1f74996dd988",
    # Add more as needed
]

# ---------- Step 2: GraphQL Query ----------
def fetch_wallet_data(wallet_address):
    url = "https://api.thegraph.com/subgraphs/name/graphprotocol/compound-v2"

    query = f"""
    {{
      account(id: "{wallet_address.lower()}") {{
        id
        tokens {{
          symbol
          supplyBalanceUnderlying
          borrowBalanceUnderlying
        }}
      }}
      liquidations(where: {{borrower: "{wallet_address.lower()}"}}) {{
        id
      }}
    }}
    """

    response = requests.post(url, json={'query': query})

    try:
        data = response.json()
        if 'data' in data:
            return data['data']
        else:
            print(f"[ERROR] GraphQL failed for {wallet_address}: {data.get('errors', 'Unknown')}")
            return None
    except Exception as e:
        print(f"[EXCEPTION] JSON parse failed for {wallet_address}: {e}")
        return None

# ---------- Step 3: Feature Extraction ----------
def extract_features(wallet_address):
    data = fetch_wallet_data(wallet_address)
    if not data:
        return {
            'wallet_id': wallet_address,
            'total_supply': 0,
            'total_borrow': 0,
            'borrow_to_supply': 0,
            'liquidation_count': 0
        }

    account = data.get('account')
    liquidations = data.get('liquidations', [])

    supply = 0
    borrow = 0

    if account and 'tokens' in account:
        for token in account['tokens']:
            try:
                supply += float(token['supplyBalanceUnderlying'])
                borrow += float(token['borrowBalanceUnderlying'])
            except:
                pass

    borrow_to_supply = borrow / supply if supply > 0 else 0

    return {
        'wallet_id': wallet_address,
        'total_supply': supply,
        'total_borrow': borrow,
        'borrow_to_supply': borrow_to_supply,
        'liquidation_count': len(liquidations)
    }

# ---------- Step 4: Process All Wallets ----------
features_list = []
for wallet in wallets:
    print(f"Processing {wallet}...")
    features = extract_features(wallet)
    features_list.append(features)
    time.sleep(0.3)  # prevent API rate-limiting

df = pd.DataFrame(features_list)

# ---------- Step 5: Normalize & Score ----------
# Handle normalization safely
df['borrow_to_supply_norm'] = (df['borrow_to_supply'] - df['borrow_to_supply'].min()) / (df['borrow_to_supply'].max() - df['borrow_to_supply'].min() + 1e-9)
df['liquidation_norm'] = (df['liquidation_count'] - df['liquidation_count'].min()) / (df['liquidation_count'].max() - df['liquidation_count'].min() + 1e-9)

# Weighted scoring: more borrow + more liquidation = more risk = lower score
df['risk_index'] = 0.7 * df['borrow_to_supply_norm'] + 0.3 * df['liquidation_norm']
df['score'] = (1 - df['risk_index']) * 1000
df['score'] = df['score'].round().astype(int)

# ---------- Step 6: Save Output ----------
df[['wallet_id', 'score']].to_csv("wallet_risk_scores.csv", index=False)
print("✅ Saved as wallet_risk_scores.csv")

# Optional: print the DataFrame
print(df[['wallet_id', 'score']])


Processing 0x0039f22efb07a647557c7c5d17854cfd6d489ef3...
[ERROR] GraphQL failed for 0x0039f22efb07a647557c7c5d17854cfd6d489ef3: [{'message': 'This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com'}]
Processing 0x06b51c6882b27cb05e712185531c1f74996dd988...
[ERROR] GraphQL failed for 0x06b51c6882b27cb05e712185531c1f74996dd988: [{'message': 'This endpoint has been removed. If you have any questions, reach out to support@thegraph.zendesk.com'}]
✅ Saved as wallet_risk_scores.csv
                                    wallet_id  score
0  0x0039f22efb07a647557c7c5d17854cfd6d489ef3   1000
1  0x06b51c6882b27cb05e712185531c1f74996dd988   1000
